In [ ]:
# Please don't change this cell, but do make sure to run it.
import otter
grader = otter.Notebook()

# Homework 5 Supplemental Notebook

## DSC 40A, Summer Session 2 2024

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Markdown

pd.options.plotting.backend = "plotly"

# DSC 40A preferred styles.
pio.templates["dsc40a"] = go.layout.Template(
    layout=dict(
        margin=dict(l=30, r=30, t=30, b=30),
        autosize=True,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        title=dict(x=0.5, xanchor="center"),
    )
)
pio.templates.default = "simple_white+dsc40a"

### Reminder

<div class="alert alert-block alert-warning" markdown="1">

This notebook **does not** need to be submitted! Instead, the problems that use it (Problem 4(f) and Problem 5(c)) will ask you to take screenshots of relevant pieces and include them in your Homework 5 PDF submission.

</div>

### Helper Functions

Here, we'll define several functions that you'll need to use in this notebook. **Don't reinvent the wheel, use the functions that are here!**

In [ ]:
def solve_normal_equations(X, y):
    '''Returns the optimal parameter vector, w*, given a design matrix X and observation vector y.'''
    return np.linalg.solve(X.T @ X, X.T @ y)

def create_design_matrix(df, columns, intercept=True):
    '''Creates a design matrix by taking the specified columns from the DataFrame df.
       Adds a column of all 1s as the first column if intercept is True, which is the default.
       The argument columns should be a list.
    '''
    df = df.copy()
    df['1'] = 1
    if intercept:
        return df[['1'] + columns].values
    else:
        return df[columns].values
    
def mean_squared_error(X, y, w):
    '''Returns the mean squared error of the predictions Xw and observations y.'''
    return np.mean((y - X @ w) ** 2)

## Problem 4: Gradient Descent Gone Wrong

<!--
BEGIN QUESTION
name: test-gradient-correct
points: 0
-->

### Problem 4(f)

First, use your answers to Problem 4(d) to complete the implementations of `R`, `dR_w0`, and `dR_w1` below, given that:
- We are trying to find the optimal parameters for the model $H(x) = w_0 + w_1 x$.
- Our dataset has two points, $(1, 5)$ and $(2, 7)$.

The function `dR` uses your implementations of `dR_w0` and `dR_w1` to find the gradient vector $\nabla R_\text{sq}(\vec{w})$; you don't need to change anything within `dR`.

Note that there is an autograder test cell below the following cell. This notebook isn't being graded, but that cell will verify that you implemented the necessary functions correctly, so that you can answer the rest of the problem.

In [ ]:
def R(w):
    w0, w1 = w
    ...
    
def dR_w0(w):
    w0, w1 = w
    ...
    
def dR_w1(w):
    w0, w1 = w
    ...
    
def dR(w):
    # Do not change the lines below.
    return np.array([dR_w0(w), dR_w1(w)])

In [ ]:
grader.check("test-gradient-correct")

Once you've implemented the functions above, run the cell below to see a graph of $R_\text{sq}(\vec{w})$. This graph is known as a **loss surface**, and it is what we are using gradient descent to minimize.

In [ ]:
num_points = 50

uvalues = np.linspace(-1, 6, num_points)
vvalues = np.linspace(-1, 6, num_points)
(u,v) = np.meshgrid(uvalues, vvalues)
thetas = np.vstack((u.flatten(),v.flatten()))

MSE = np.array([R(t) for t in thetas.T])

loss_surface = go.Surface(x=u, y=v, z=np.reshape(MSE, u.shape))

fig = go.Figure(data=[loss_surface])

fig.update_layout(scene = dict(
    xaxis_title = "w0",
    yaxis_title = "w1",
    zaxis_title = r"R(w0, w1)"))

fig.show()

Now, the function below uses `dR` to implement gradient descent. What it outputs is the same loss surface as above, along with a visualization of the **path** gradient descent took to find the minimizer, $\vec{w}^*$. Each point corresponds to a different guess, $\vec{w}^{(i)}$, and consecutive guesses are connected with a line.

In [ ]:
def minimize_R(initial, alpha):
    w_history = []
    risk_history = []
    w = np.array(initial)
    count = 0
    
    # The important part is here!
    while np.linalg.norm(dR(w)) >= 0.01:
        w_history.append(w)
        risk_history.append(R(w))
        w = w - alpha * dR(w)
        count += 1
        
    display(Markdown(r'Best $\vec{w}^* = [' + ','.join(w.round(3).astype(str)) + "]^T$"))
    display(Markdown(f'Iterations required: {count}'))
    # ---
    
    # Plot path.
    scatter = go.Scatter3d(
        x=[w_history[i][0] for i in range(len(risk_history))], 
        y=[w_history[i][1] for i in range(len(risk_history))], 
        z=risk_history, 
        mode="lines+markers",                   
        marker=dict(
            symbol="circle",
            size=8)
    )

    fig2 = go.Figure(data=[fig.data[0], scatter])
    return fig2
    # ---

<span style="color:red"><b>Your Job</b></span>

First, run the cell below. It visualizes the execution of gradient descent with $\vec{w}^{(0)} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}$ and $\alpha = 0.1$. Then, repeatedly run the cell below, but change the values of `initial` and `alpha`. (Run it with at least 20 different combinations of arguments – what happens when you make `alpha` bigger? Smaller? What if you change `initial`?)

**In your PDF**, answer the following questions:

- Around what value of $\alpha$ does gradient descent stop converging? Include a screenshot of the plot below with the largest value of $\alpha$ you were able to successfully find $\vec{w}^*$ with.
- Why isn't gradient descent showing "Best $\vec{w}^* = [3, 2]^T$", but instead a slightly different $\vec{w}^*$, even though we know $w_0^* = 3$ and $w_1^* = 2$ exactly?

In [ ]:
minimize_R([0, 0], 0.1)

Remember, you do not need to submit this notebook! Instead, follow the instructions in the <span style="color:red"><b>Your Job</b></span> cell above.

## Problem 5: Transformation Tuesday

Here are two new functions – the logistic function, `sigma`, and its inverse, `sigma_inv`. Note that both functions are **vectorized**, meaning that if you give them an array as input, you will get an array back as output.

In [ ]:
def sigma(x):
    return 1 / (1 + np.e ** (-x))

def sigma_inv(x):
    return np.log((x / (1 - x)))

Now, let's load in our data.

In [ ]:
logistic_data = pd.read_csv('data/logistic_data.csv')
logistic_data

Here's a scatter plot of it:

In [ ]:
px.scatter(logistic_data, x='x', y='y')

### Problem 5(c)

In the cell below, assign `X` and `z` to the design matrix and observation vector such that `solve_normal_equations(X, z)` will give us our optimal parameter vector.

_Hint: Remember that `sigma` and `sigma_inv` are vectorized, as noted above, and that we've defined `create_design_matrix` for you. Each definition should only take one line. **Don't use a `for`-loop!**_

In [ ]:
X = ...
X

In [ ]:
z = ...
z

Once you answer the above two parts, you should be able to run the following two cells to see the value of $\vec{w}^*$ and a fit version of our hypothesis function.

In [ ]:
w_star_logistic = solve_normal_equations(X, z)
w_star_logistic

In [ ]:
x_range = np.linspace(-11, 5)

fig = go.Figure()
fig.add_trace(go.Scatter(x=logistic_data['x'], y=logistic_data['y'], mode='markers', name='actual'))
fig.add_trace(go.Scatter(x=x_range, 
                         y=sigma(w_star_logistic[0] + w_star_logistic[1] * x_range), 
                         name='Hypothesis Function', 
                         line=dict(color='red')))

fig.update_layout(xaxis_title='Total Bill', yaxis_title ='Tip')

Remember, you do not have to submit this notebook! Instead, include a screenshot of the above plot in your PDF for Problem 5(c).